<h2>Deep Convolutional Generative Adversarial Nets (DCGAN) - generate CelebA faces</h2>

<p>Original DCGAN paper: <a href='https://arxiv.org/pdf/1511.06434.pdf'>https://arxiv.org/pdf/1511.06434.pdf</a></p>

<p>The model structure is borrowed from and modified based on Arthur Juliani's implementation: <a href='https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.4kvc6juwc'>https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.4kvc6juwc</a></p>

In [12]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import time
from PIL import Image
from images2gif import writeGif

## Configuration

In [2]:
tf.reset_default_graph()
np.random.seed(1) 
tf.set_random_seed(1)

num_train_imgs = 10000 # number of training images         
original_img_directory = './celeba_align_178_218/'
resized_img_directory = 'celeba_resizesd/'

pre_train_epochs = 1 # num of epochs to pre-train discriminator
training_epochs = 20001 # num of epochs to train discriminator and generator jointly
batch_size = 64 # batch size for training the model, must be larger than batch_size_sample

# for the generated images
batch_size_sample = 36 # number of images to be generated for viewing the training progress
# the generated images will be concatenated, the product of the following must equal to batch_size_sample
sample_num_rows = 6 # how many rows to have
sample_num_columns = 6 # how many columns to have

leak = 0.2 # degree of leakiness used in leaky ReLU
alpha = 0.0002 # base learning rate
beta1 = 0.5 # the fraction factor used in the first momentum term from Adam optimizer
k = 1 # this is number of times to update generator for every time discriminator is updated in each epoch
logs_path = "./dcgan_face_log1" # directory to save the training log to
train_sample_directory = './dcgan_face/train_sample/' # directory to save the generated images during training
model_directory = './dcgan_face/models' # directory to save trained model
sample_directory = './dcgan_face/generated_sample/' # directory to save the generated images

<h3>Just once - Preprocess training images by resizing them</h3>
<p>Face images can be found at: <a href="http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html">http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html</a></p>

In [ ]:
# get a list of image filenames
file_names = [f for f in os.listdir(original_img_directory)[:num_train_imgs+1] if f.endswith('.jpg')] 
# get a list of images
images = []
for f in file_names:
    img = Image.open(original_img_directory+f)
    images.append(img.copy())t
    img.close() 

if not os.path.exists(resized_img_directory):
    os.makedirs(resized_img_directory)

# save the resized images
for i,img in enumerate(images):
    img = resizeimage.resize_contain(img, [64, 64]) # resize the image from 178*218 to 64*64
    img.save(resized_img_directory+'/'+file_names[i], img.format)

In [ ]:
del images # remove images variable to free memory

## Read training images

In [172]:
# get a list of image filenames
file_names = [f for f in os.listdir(resized_img_directory) if f.endswith('.jpg')] 
# get a list of images
images = []
for f in file_names:
    img = Image.open(resized_img_directory+f)
    images.append(img.copy())
    #img.close() 
    
# turn image into ndarray
train_images = np.array([np.asarray(img) for img in images])

## Declare helpful functions

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def leaky_relu(x, leak=leak, name="leaky_relu"):
    with tf.variable_scope(name):
        return tf.maximum(leak*x, x)

# get a batch of real images, change size from 28*28 to 32*32, return the processed batch
def get_x(batch_size):
    indices = np.random.randint(num_train_imgs, size=batch_size) # random sample real images
    batch = train_images[indices] 
    batch = 2*(batch/255.)-1 # change range from [0, 255] to [-1, 1]
    return batch

# wrapper function for real images
def save_real_images(images, size, image_path):
    concat_img = concat(images, size) # concatenate individual images to a grid of images
    concat_img = Image.fromarray(concat_img) # convert ndarray to an image object
    concat_img.save(image_path, 'PNG')

# wrapper function for generated images
def save_generated_images(images, size, image_path):
    images = inverse_transform(images)
    concat_img = concat(images, size) # concatenate individual images to a grid of images
    concat_img = Image.fromarray(concat_img) # convert ndarray to an image object
    concat_img.save(image_path, 'PNG')

# change values from [-1, 1] to [0, 255]
def inverse_transform(images):
    return (images+1.)/2. * 255

# concatenate individual images to a grid of images
def concat(images, size):
    # get height and width for a single image generated, e.g. 64 * 64
    height, width = images.shape[1], images.shape[2] 
    
    # placeholder for a concatenated img, which have 
    # img_height = image height * num images vertically, e.g. 64 * 5
    # img_width = image width * num images horizontally, e.g. 64 * 5
    img = np.zeros((height * size[0], width * size[1],3)) 

    # loop through each image
    for index, image in enumerate(images):
        j = index / size[0] # image row index
        i = index % size[1] # image column index
        img[j*height:j*height+height, i*width:i*width+width,:] = image

    return img.astype(np.uint8) # convert ndarray to uint8 type

### Save orginal images

In [ ]:
if not os.path.exists(train_sample_directory):
    os.makedirs(train_sample_directory)

indices = np.random.randint(num_train_imgs, size=batch_size_sample) # random sample real images
batch = train_images[indices] # with shape [batch,64,64,3]
save_real_images(images=batch, size=[sample_num_rows,sample_num_columns], image_path=train_sample_directory+'/fig_original.png')

## Define the Generative Adversarial Network

#### Generator network

In [4]:
# The generator network takes a noise vector z and return a 64 * 64 image. 
# We use tensorflow slim to perform fractionally-strided convolution, batch normalization and ReLU activation
# for each layer except the last layer
def generator(z):
    
    # Turn z into the first tensor + batch norm + relu
    # Creates a fully connected weight matrix, which is multiplied by the mini-batch of z vectors to 
    # produce a hidden layer with 4*4*1024 hidden nodes
    zP = slim.fully_connected(inputs=z,num_outputs=4*4*1024,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_init',weights_initializer=initializer)
    # Transform the flat 4*4*512 layer into a tensor, whose kernel size is 4*4 and 512 kernels in total
    # -1 stands for the mini-batch size that's to be computed
    zCon = tf.reshape(zP,[-1,4,4,1024])
    
    # Perform fractionally-strided convolution/deconvolution + batch norm + relu
    # num_outputs: number of output filters/activation maps
    # kernel_size: [kernel_height, kernel_width]
    # stride: [stride_height, stride_width], in this case it is generating four pixels out of every pixel, this gives fractionally-strided convolution
    # output size batch_size*8*8*512
    gen1 = slim.convolution2d_transpose(\
        inputs=zCon,num_outputs=512,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    # output size batch_size*16*16*256
    gen2 = slim.convolution2d_transpose(\
        inputs=gen1,num_outputs=256,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    # output size batch_size*32*32*128
    gen3 = slim.convolution2d_transpose(\
        inputs=gen2,num_outputs=128,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    
    # output size batch_size*64*64*3
    g_out = slim.convolution2d_transpose(\
        inputs=gen3,num_outputs=3,kernel_size=[5,5],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_output', weights_initializer=initializer)
    
    return g_out

### Discriminator network

In [5]:
# The discriminator network takes a 64*64 image and return a probability of whether it is real or generated
# We use tensorflow slim to perform standard convolution, batch normalization and leaky ReLU activation
# for each layer
def discriminator(image, reuse=False):
    # Creates 32 4*4 filters to convolve on the mini-batch of 64*64*3 images, also perform batch norm + leaky ReLU activation
    # Note that no pooling is performed
    # stride here calculates one pixel out of every 2*2 pixels, this gives strided convolution that will shrink the image as a substitute for pooling
    # Set reuse=True allows discriminator to evaluate both real samples and generated samples 
    # Outputs size batch_size*32*32*32 
    
    dis1 = slim.convolution2d(inputs=image,num_outputs=32,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    # outputs size batch_size*16*16*64 
    dis2 = slim.convolution2d(inputs=dis1,num_outputs=64,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    # outputs size batch_size*8*8*128 
    dis3 = slim.convolution2d(inputs=dis2,num_outputs=128,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    # outputs size batch_size*4*4*256 
    dis4 = slim.convolution2d(inputs=dis3,num_outputs=256,kernel_size=[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=leaky_relu,\
        reuse=reuse,scope='d_conv4',weights_initializer=initializer)

    # flatten the tensor to [batch_size, 4*4*256]
    dis_flat = slim.flatten(dis4)
    
    # create a fully connect layer with dis_flat and just one node in the output layer
    # note there's no batch normalization at this layer
    # outputs size batch_size*1 
    d_out = slim.fully_connected(inputs=dis_flat,num_outputs=1,\
        activation_fn=tf.nn.sigmoid, reuse=reuse, scope='d_output', weights_initializer=initializer)
    
    return d_out

## Connecting two networks together

In [ ]:
z_size = 100 # size of initial noise vector that will be used for generator

# initialize all parameters of the networks
# weights were initialized from a zero-centered Normal distribution with standard deviation 0.02
# tf.truncated_normal returns random values from a normal distribution and made sure no value exceeds 2 std
initializer = tf.truncated_normal_initializer(stddev=0.02)

# placeholders for inputs into the generator and discriminator, respectively.
z_vector = tf.placeholder(shape=[batch_size,z_size],dtype=tf.float32, name='z_vectors') 
x_vector = tf.placeholder(shape=[batch_size,64,64,3],dtype=tf.float32, name='real_images') 


# ---- Pre-training ----
# the discriminator should output probability=1 for all the training images
train_labels=tf.constant(1.0,shape=(batch_size,1), name='pre_train_labels')
# feed images to the discriminator and return the predicted probability
d_pre_output = discriminator(x_vector)                                              
summary_pre_d_x_hist = tf.histogram_summary("pre_train_d_prob_x", d_pre_output)

d_pre_loss=tf.reduce_mean(tf.square(d_pre_output-train_labels))
summary_pre_d_loss = tf.scalar_summary("pre_train_d_loss", d_pre_loss)
# ---- end of Pre-training ----


# ---- DCGAN ----
g_output = generator(z_vector) # generated mini-batch of images from noisy z vectors 
    
d_output_x = discriminator(x_vector,reuse=True) # probabilities for real images
d_output_x = tf.maximum(tf.minimum(d_output_x, 0.99), 0.01) # avoid inf and -inf
summary_d_x_hist = tf.histogram_summary("d_prob_x", d_output_x)

d_output_z = discriminator(g_output,reuse=True) # probabilities for generated images
d_output_z = tf.maximum(tf.minimum(d_output_z, 0.99), 0.01) # avoid inf and -inf
summary_d_z_hist = tf.histogram_summary("d_prob_z", d_output_z)

d_loss = -tf.reduce_mean(tf.log(d_output_x) + tf.log(1-d_output_z)) # loss for discriminator
summary_d_loss = tf.scalar_summary("d_loss", d_loss)

g_loss = -tf.reduce_mean(tf.log(d_output_z)) # loss for generator
summary_g_loss = tf.scalar_summary("g_loss", g_loss)

# the following parameter indices may change if the network structure changes
para_d = tf.trainable_variables()[:9] # parameters for discriminator
para_g = tf.trainable_variables()[9:] # parameters for generator

# only update parameters in discriminator during pre-training
pre_optimizer = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(d_pre_loss,var_list=para_d)
# only update the weights for the discriminator network
optimizer_op_d = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(d_loss,var_list=para_d)
# only update the weights for the generator network
optimizer_op_g = tf.train.AdamOptimizer(learning_rate=alpha,beta1=beta1).minimize(g_loss,var_list=para_g)

## Train the DCGAN model

In [ ]:
# create a log folder and save the graph structure, do this before training
g_writer = tf.train.SummaryWriter(logs_path + '/generator', graph=tf.get_default_graph())
d_writer = tf.train.SummaryWriter(logs_path + '/discriminator')

# saver saves and loads variables of the model to and from checkpoints, 
# which are binary files that maps variable names to tensor values
saver = tf.train.Saver(max_to_keep=20) 


with tf.Session(config=config) as sess:  
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())
    #print [v.name for v in tf.trainable_variables()] # print all variable names
    
    
    # -------- pre-train discriminator --------
    start = time.time()
    for epoch in range(pre_train_epochs):
        x = get_x(batch_size) # get a batch of real images, with range [-1 ,1]
        
        d_pre_summary_merge = tf.merge_summary([summary_pre_d_loss, summary_pre_d_x_hist])
        summary_pre_d,_=sess.run([d_pre_summary_merge,pre_optimizer], feed_dict={x_vector: x})
        
        d_writer.add_summary(summary_pre_d, epoch) # add loss summary to tensorboard
        time_lapse = time.time()-start
        start = time.time()
        print "pre-train epoch: ", epoch,", time spent: %.2fs" % time_lapse
   
    print "pre-train done."
    
    
    
    # -------- jointly training discriminator and generator --------
    start = time.time()
    
    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
    
    for epoch in range(training_epochs):
        # get a batch of real images, with range [-1 ,1]
        x = get_x(batch_size) 
        # mini-batch of noise data from [-1, 1]
        z = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        
        
        if epoch <= 200:
            # make a directory for generated images
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            # get a generated image, with range [-1, 1]
            g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) 
            # substitute 1/5 of the training images to the generated images, to increase discriminator's difficulty 
            substitute_indices = np.random.randint(batch_size, size=batch_size/5) 
            x[substitute_indices] = g_images[substitute_indices]

        
        # Update the discriminator
        d_summary_merge = tf.merge_summary([summary_d_loss, summary_d_x_hist,summary_d_z_hist])
        summary_d,_ = sess.run([d_summary_merge,optimizer_op_d],feed_dict={z_vector:z, x_vector:x}) 
        
        
        if epoch<20:
            # Update the generator for k times in the specified n epoch
            for i in range(k):
                summary_g,_ = sess.run([summary_g_loss,optimizer_op_g],feed_dict={z_vector:z})
        else:
            summary_g,_ = sess.run([summary_g_loss,optimizer_op_g],feed_dict={z_vector:z})
        
        
        # add loss summary to tensorboard
        if epoch % 1 == 0:
            d_writer.add_summary(summary_d, epoch) 
            g_writer.add_summary(summary_g, epoch)
        
        # output generated image
        if epoch % 200 == 0 or epoch in [5,60,100,150]:
            time_lapse = time.time()-start
            start = time.time()
            
            print "DCGAN epoch: ", epoch,", time spent: %.2fs" % time_lapse
            g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) # get a generated image, with range [-1, 1]
            
            # make a directory for generated images
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            
            #Save sample generator images for viewing training progress.
            save_generated_images(images = np.reshape(g_images[0:batch_size_sample],[batch_size_sample,64,64,3]),\
                        size = [sample_num_rows,sample_num_columns], image_path = train_sample_directory+'/'+str(epoch)+'.png')
            
        if epoch in [0,500,1000,2000,4000,6000,8000,10000,15000]:
            # make a directory for trained models
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            
            # save the trained model at different epoch
            saver.save(sess, save_path = model_directory + '/' + str(epoch) + '.cptk')
    print "Done"

### Continue training from the last checkpoint

In [ ]:
saver = tf.train.Saver(max_to_keep=20)

In [ ]:
ckpt = tf.train.get_checkpoint_state(model_directory)
ckpt.model_checkpoint_path

In [ ]:
print 'Loading models...might take a minute'
saver = tf.train.Saver(max_to_keep=20)

# create a log folder and save the graph structure, do this before training
g_writer = tf.train.SummaryWriter(logs_path + '/generator', graph=tf.get_default_graph())
d_writer = tf.train.SummaryWriter(logs_path + '/discriminator')

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:  
    start = time.time()
    
    ckpt = tf.train.get_checkpoint_state(model_directory)
    model = ckpt.model_checkpoint_path

    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
    
    saver.restore(sess, save_path=model)
    
    for epoch in range(14500,training_epochs):
        # get a batch of real images, with range [-1 ,1]
        x = get_x(batch_size) 
        # mini-batch of noise data from [-1, 1]
        z = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        
        # Update the discriminator
        d_summary_merge = tf.merge_summary([summary_d_loss, summary_d_x_hist,summary_d_z_hist])
        summary_d,_ = sess.run([d_summary_merge,optimizer_op_d],feed_dict={z_vector:z, x_vector:x}) 
        
        # Update the generator
        summary_g,_ = sess.run([summary_g_loss,optimizer_op_g],feed_dict={z_vector:z})
        
        # add loss summary to tensorboard
        if epoch % 20 == 0:
            d_writer.add_summary(summary_d, epoch) 
            g_writer.add_summary(summary_g, epoch)
        
        # output generated image
        if epoch % 200 == 0:
            time_lapse = time.time()-start
            start = time.time()
            
            print "DCGAN epoch: ", epoch,", time spent: %.2fs" % time_lapse
            g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) # get a generated image, with range [-1, 1]
            
            # make a directory for generated images
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            
            #Save sample generator images for viewing training progress.
            save_generated_images(images = np.reshape(g_images[0:batch_size_sample],[batch_size_sample,64,64,3]),\
                        size = [sample_num_rows,sample_num_columns], image_path = train_sample_directory+'/'+str(epoch)+'.png')
            
        if epoch in [16500,17000]:
            # make a directory for trained models
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            
            # save the trained model at different epoch
            saver.save(sess, save_path = model_directory + '/' + str(epoch) + '.cptk')
    print "Done"

### Combine sample images generated from training and save it as GIF

In [ ]:
file_names = sorted([int(f[:-4]) for f in os.listdir(train_sample_directory) if f.endswith('.png')]) # get image filenames
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(train_sample_directory+f) for f in file_names] # turn into image instances
filename = "train_samples.gif"

if not os.path.exists(train_sample_directory):
            os.makedirs(train_sample_directory)
writeGif(train_sample_directory+filename, images, duration=0.1) # combine images to gif

## Use the trained generator to generate images

In [ ]:
start = time.time()

print 'Loading models...might take a minute'
saver = tf.train.Saver(max_to_keep=10)

with tf.Session() as sess:
    #sess.run(tf.initialize_all_variables())
    
    # reload the trained model.
    ckpt = tf.train.get_checkpoint_state(model_directory)
    models = ckpt.all_model_checkpoint_paths
    #print 'all saved models: ',ckpt.all_model_checkpoint_paths
    
    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
    
    # loop through each checkpoint and use models to generate images at different training stages
    for index, model in enumerate(models):
        saver.restore(sess, save_path=model)

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample}) # get a generated image

        # make a directory for generated images
        if not os.path.exists(sample_directory):
            os.makedirs(sample_directory)

        #Save generated sample images
        save_generated_images(images = np.reshape(g_images[0:batch_size_sample],[batch_size_sample,64,64,3]),\
                    size = [sample_num_rows,sample_num_columns], image_path = sample_directory+'/'+str(index)+'.png')

        print "Model "+str(index),': ',model
        
time_lapse = time.time()-start
print " Time spent: %.2fs" % time_lapse

## Turn generated images from checkpoints into an animated GIF

<p>Restart the kernel if not working..</p>

In [ ]:
file_names = sorted([int(f[:-4]) for f in os.listdir('.') if f.endswith('.png')]) # get sorted image filenames 
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(f) for f in file_names] # turn into image instances
filename = "train_samples.gif"
writeGif(filename, images, duration=0.1) # combine images to gif

## Interpolation Preparation

In [6]:
model_directory = './dcgan_face_15k/models/' # directory to save trained model
sample_inter_directory = './dcgan_face_15k/interpolation/' # directory to save the generated images from linear interpolation
# make a directory for generated images
if not os.path.exists(sample_inter_directory):
    os.makedirs(sample_inter_directory)

In [7]:
# spherical interpolation between pointA and pointB, each is a ndarray
# val is a value between [0,1], where 0 returns pointA, 1 returns pointB
def slerp(val, pointA, pointB):
    omega = np.arccos(np.clip(np.dot(pointA/np.linalg.norm(pointA), pointB/np.linalg.norm(pointB)), -1, 1))
    so = np.sin(omega)
    if so == 0:
        return (1.0-val) * pointA + val * pointB # L'Hopital's rule/LERP
    return np.sin((1.0-val)*omega) / so * pointA + np.sin(val*omega) / so * pointB

In [8]:
tf.reset_default_graph()
# size of initial noise vector that will be used for generator
z_size = 100 

# placeholders for inputs into the generator
# shape[0] is None means it could take any integer value
z_vector = tf.placeholder(shape=[None,z_size],dtype=tf.float32, name='z_vectors')

# initialize all parameters of the networks
# weights were initialized from a zero-centered Normal distribution with standard deviation 0.02
# tf.truncated_normal returns random values from a normal distribution and made sure no value exceeds 2 std
initializer = tf.truncated_normal_initializer(stddev=0.02)

# generated mini-batch of images from noisy z vectors
g_output = generator(z_vector)

print 'Loading Face models...might take a minute'
saver = tf.train.Saver()

Loading Face models...might take a minute


## Interpolation

In [64]:
with tf.Session() as sess:

    # reload the trained model.
    ckpt = tf.train.get_checkpoint_state(model_directory)
    model = ckpt.model_checkpoint_path # just read the model at epoch 16500
    print 'most trained model: ', model

    # z noise vector that will be used to generate image to check the training progress
    z_sample = np.random.uniform(-1.0,1.0,size=[batch_size_sample,z_size]).astype(np.float32)

    # restore model variables
    saver.restore(sess, save_path=model)

    # linear interpolation
    z_sample_one = z_sample[:2]
    #print z_sample_one.shape

    # set fixed input, interpolate from 7 to 9
    sample1='''0.3904759   0.12369447 -0.03130473 -0.77256024  0.54540509 -0.63576019
   0.64418173  0.0961884  -0.30221939  0.7397421  -0.41891637  0.23050123
  -0.85202283  0.09574939  0.49240941  0.84485519 -0.4560945  -0.73232478
   0.88528788 -0.49584585 -0.05611671 -0.94211501  0.80680168 -0.62426782
  -0.18791643  0.67589283 -0.06083585 -0.19920608 -0.8094129  -0.3279312
   0.99031818  0.72150308 -0.66078311 -0.81424487 -0.60490525 -0.71164876
  -0.0634105   0.02162064  0.56584495  0.09616423  0.51447213  0.09199435
  -0.5390138   0.02217139 -0.90989834  0.64157981  0.17594992 -0.42963964
  -0.59015018  0.83339399  0.2526193  -0.78287238  0.50538272 -0.51408827
   0.9690786  -0.90271968  0.93069273 -0.86799151 -0.86437082 -0.39811093
   0.06549309 -0.84584093  0.52733558 -0.90270567  0.41961735  0.62828648
  -0.81002378  0.53971922  0.14811444 -0.93527913 -0.51290333 -0.48542583
   0.74688935  0.4627139   0.54627806 -0.14855324  0.5601418   0.24667268
  -0.413488    0.46806765  0.78500813  0.64971685  0.63234931 -0.28815132
  -0.42552331  0.40306959 -0.71005774 -0.44297776 -0.76836979  0.24986272
  -0.02224439  0.65198964 -0.97852767  0.46914703 -0.8889237  -0.39507776
  -0.93041956 -0.88002688 -0.24425523  0.41087016'''
    sample2=''' 0.54349154  0.82320267 -0.62965882  0.98068064 -0.56107283 -0.31114981
  -0.45138538  0.80631655  0.90001726 -0.9619692   0.93649179  0.67718279
   0.13684638 -0.15676339 -0.3471978  -0.83544356  0.24256505 -0.42204261
  -0.4512648   0.16144913 -0.82650888  0.07874123 -0.49977717 -0.1294433
  -0.8050105   0.44765326  0.39748943  0.84955609 -0.97319156  0.3089633
  -0.34072471  0.14846873 -0.08748962  0.14348175  0.03980778  0.03936955
   0.11320399 -0.65704525 -0.23477662 -0.94302213  0.45144475  0.05105386
  -0.11944827  0.64141428 -0.9440648  -0.7251038  -0.04335017  0.45072994
  -0.062515   -0.97800612  0.25549379  0.43639576 -0.00373689 -0.48526773
  -0.85005528 -0.71420103  0.16092534 -0.34909213 -0.72042316 -0.85552001
  -0.0950563   0.64399147 -0.14060096 -0.29917473  0.39624763 -0.94713563
  -0.63286591  0.49637678 -0.2596755  -0.06869141 -0.46832797 -0.86966354
  -0.87580574 -0.08289656 -0.35145256 -0.47775638  0.22160572 -0.92910951
  -0.05955737 -0.92531306  0.65753967  0.195959   -0.3773948  -0.6927458
   0.47357163  0.39965707  0.75121379 -0.83463138  0.92006731  0.96232468
   0.14594369 -0.20597722  0.16611969 -0.56408048  0.3470183  -0.48169822
  -0.74719411 -0.9993121  -0.33036932  0.7356177 '''
    # turn the string above into an array. sample1 represents a point in a 100-dimensional space
    #sample1 = np.array([float(i.rstrip()) for i in sample1.split(' ') if i != ''])
    sample1 = np.array(z_sample[0])
    #sample2 = np.array([float(i.rstrip()) for i in sample2.split(' ') if i != ''])
    sample2 = np.array(z_sample[1])
    #print z_sample[:2]
    z_sample_one[1] = np.zeros(100) # keep the second face constant
    
    num_inter = 9 # number of interpolated images
    
    print 'one-dimension interpolation'
    dim = 0 # only change values in this dimension
    
    '''
    # transition from one face to the face in the middle, then to another face, by changing value in one dimension
    for i, value in enumerate(np.linspace(-20,20,num_inter)): 
    # transition from the face in the middle to another face
    #for i, value in enumerate(np.linspace(0,20,num_inter)):  
        z_sample_one[0][dim] = value 
        #print z_sample_one
        g_images = sess.run(g_output,feed_dict={z_vector:z_sample_one})
        g_image = g_images[:1] # only retrieve the first returned image

        # save interpolated image
        save_generated_images(images = g_image,\
            size = [1,1], image_path = sample_inter_directory+'/'+str(i)+'.png')
    '''
    
    '''
    print 'linear interpolation'
    # linear interpolation between sample1 and sample2            
    for i in range(num_inter):
        if i == 0: # interpolation start with point A
            z_sample_one[0] = sample1
        elif i == num_inter-1: # interpolation end with point B
            z_sample_one[0] = sample2
        else:
            z_sample_one[0] += (sample2-sample1)/num_inter # linear interpolate from point A to B
        #print z_sample_one,'\n'

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample_one})
        g_image = g_images[:1] # only retrieve the first returned image
        #print g_image.shape
            
        # save interpolated image
        save_generated_images(images = g_image,\
            size = [1,1], image_path = sample_inter_directory+'/'+str(i)+'.png')
    '''
    
    
    print 'spherial interpolation'
    # spherical interpolation between pointA and pointB, each is a ndarray
    # val is a value between [0,1], where 0 returns pointA, 1 returns pointB
    for i, value in enumerate(np.linspace(0,1,num_inter)):
        z_sample_one[0] = slerp(val=value, pointA=sample1, pointB=sample2) 
        #print z_sample_one,'\n'

        g_images = sess.run(g_output,feed_dict={z_vector:z_sample_one})
        g_image = g_images[:1] # only retrieve the first returned image

        # save interpolated image
        save_generated_images(images = g_image,\
            size = [1,1], image_path = sample_inter_directory+'/'+str(i)+'.png')
    
    print 'done'

most trained model:  ./dcgan_face_15k/models/16500.cptk
one-dimension interpolation
spherial interpolation
done


## turn the interpolate images into GIF

In [33]:
file_names = sorted([int(f[:-4]) for f in os.listdir(sample_inter_directory) if f.endswith('.png')]) # get image filenames
file_names = [str(f)+'.png' for f in file_names] # add the format suffix at the end of the filenames
images = [Image.open(sample_inter_directory+f) for f in file_names] # turn into image instances
filename = "aa.gif"
writeGif(sample_inter_directory+filename, images, duration=0.1) # combine images to gif